In [1]:
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy

from utils.reproduce_functions import *
from utils.mutation_functions import *
from utils.parent_selection_functions import *
from utils.other_fucntions import *
from utils.valhalla_functions import *

import lab9_lib

In [2]:
PROBLEM_SIZE = 10

MU = 10
LAMBDA = 20
strategy = 'plus' # comma or plus

MUTATION_PROB = 0.2
DYNAMIC_MUTATION_PROB = True
DIVERSITY_THRESHOLD = 20

LENGTH_SOLUTION = 1000
NUMBER_GENERATIONS = 1_500

EPOCH = 10              # Number of generation to reintroduce the best fitting
N_CHAMPIONS = 2        # Number of champions that will rest in the Valhalla
N_ISLANDS = 1                       # Number of different populations
N_MIGRANTS = 5                      # Number of migrants from one island
SIZE_VALHALLA = 3 * N_CHAMPIONS 

In [3]:
mutate = one_bit_flip
reproduce = random_xover
parent_selection = roulette
when_to_pick = fixed_generations
when_load_valhalla = fixed_generations

In [4]:
def ga(fitness, parents, parents_evals, memoization=False):
    generation_convergence = -1
    valhalla = np.empty(shape = ((EPOCH // 10) * N_CHAMPIONS, LENGTH_SOLUTION), dtype=np.int16)   
    valhalla_evals = np.empty(shape = ((EPOCH // 10) * N_CHAMPIONS, ), dtype=np.int16)   
    
    if memoization:
        pop_history = {}
        for i in range(len(parents)):
            pop_history[parents[i].tobytes()] = parents_evals[i]

    for generation in tqdm(range(NUMBER_GENERATIONS)):
        offsprings = []
        offsprings_evals = []
        while len(offsprings) < LAMBDA:
            # Parent Selection
            p1, p2 = parent_selection(parents, parents_evals)

            # Reproduce Parents
            off_spring = reproduce(p1, p2)

            # Mutate Offspring
            if DYNAMIC_MUTATION_PROB:
                p_div = get_parents_diversity(p1, p2)
                new_ind1 = mutate(off_spring, \
                                  mutation_probability=(1 - (min(p_div,LENGTH_SOLUTION/2)/(LENGTH_SOLUTION/2))) )
            else:    
                new_ind1 = mutate(off_spring, mutation_probability=MUTATION_PROB)

            # Evaluate Offspring
            if memoization:
                new_ind1_eval = pop_history.get(new_ind1.tobytes())
                if new_ind1_eval == None:
                    new_ind1_eval = fitness(new_ind1)
                    pop_history[new_ind1.tobytes()] = new_ind1_eval
            else:
                new_ind1_eval = fitness(new_ind1)

            # Add it to new population
            offsprings.append(new_ind1)
            offsprings_evals.append(new_ind1_eval)


        # Create new population (with parents if plus, without if comma)
        all_people = (parents if strategy=='plus' else []) + offsprings
        all_evals = (parents_evals if strategy=='plus' else []) + offsprings_evals
        best_people = np.argsort(all_evals)[::-1]

        parents = []
        parents_evals = []
        for i in range(MU):
            parents.append(all_people[best_people[i]])
            parents_evals.append(all_evals[best_people[i]])
            
        if np.max(parents_evals) - 1.0 >= 0 and generation_convergence == -1:
            generation_convergence = generation, fitness.calls

        if when_to_pick(generation, 10):
            offsprings = np.array(offsprings)
            offsprings_evals = np.array(offsprings_evals)
            champions = offsprings[np.argsort(offsprings_evals)[-N_CHAMPIONS:]]
            start = (((generation - 1) % EPOCH) // 10) * N_CHAMPIONS
            end = (((generation - 1) % EPOCH) // 10) * N_CHAMPIONS + N_CHAMPIONS
            valhalla[start:end, :] = offsprings[np.argsort(offsprings_evals)[-N_CHAMPIONS:]]
            valhalla_evals[start:end] = offsprings_evals[np.argsort(offsprings_evals)[-N_CHAMPIONS:]]

        if when_load_valhalla(generation, EPOCH):
            parents = list(np.concatenate((parents, valhalla)))
            parents_evals = list(np.concatenate((parents_evals, valhalla_evals)))
            # # Introduce some randomicity in order to inject champions from time to time.
            # if random.random() < 0.8:
            #     parents = np.concatenate((parents, random.choices(valhalla, k = 4)))

    return parents, parents_evals

Without memoization

In [5]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
parents = [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(MU)]
parents_evals = [fitness(x) for x in parents]
parents, parents_evals = ga(fitness, parents, parents_evals, memoization=False)
i_best = np.argmax(parents_evals)
# print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)

  0%|          | 0/1500 [00:00<?, ?it/s]

100%|██████████| 1500/1500 [00:16<00:00, 92.21it/s]

Best score:  0.18
Num fitness calls:  30010


With memoization

In [6]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
parents = [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(50)]
parents_evals = [fitness(x) for x in parents]
parents, parents_evals = ga(fitness, parents, parents_evals, memoization=True)
i_best = np.argmax(parents_evals)
# print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)

100%|██████████| 1500/1500 [00:17<00:00, 84.51it/s] 


Best score:  0.275
Num fitness calls:  23516
